# Part I - ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

# Helper function:
def pandas_dataframe(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

    # Reading the csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        # Skip header row
        next(csvreader)
        
        # Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('There are {} rows '.format(sum(1 for line in f)))

There are 6821 rows 


In [5]:
# Read file into df
file = 'event_datafile_new.csv'
df = pd.read_csv(file, encoding='utf8')
print(df.head())

              artist firstName gender  itemInSession lastName     length  \
0  Sydney Youngblood     Jacob      M             53    Klein  238.07955   
1         Gang Starr     Layla      F             88  Griffin  151.92771   
2              3OH!3     Layla      F             89  Griffin  192.52200   
3        RÃÂ¶yksopp     Jacob      M             54    Klein  369.81506   
4         Kajagoogoo     Layla      F             90  Griffin  223.55546   

  level                             location  sessionId  \
0  paid  Tampa-St. Petersburg-Clearwater, FL        954   
1  paid         Lake Havasu City-Kingman, AZ        984   
2  paid         Lake Havasu City-Kingman, AZ        984   
3  paid  Tampa-St. Petersburg-Clearwater, FL        954   
4  paid         Lake Havasu City-Kingman, AZ        984   

                                          song  userId  
0                            Ain't No Sunshine      73  
1                   My Advice 2 You (Explicit)      24  
2  My First Kiss 

# Part II - Apache Cassandra database modeling

#### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Creating Cluster and Keyspace

#### Creating a Cluster

In [6]:
# Connect to a Cassandra instance on the local machine
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# Create session to establish the connection and begin executing queries
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create sparkify keyspace(database)
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}
""")

#### Set Keyspace

In [8]:
# Set KEYSPACE to sparkify
session.set_keyspace('sparkify')

## Create queries that we will model our non-relational databse to

#### Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession  = 4

For this query, we will create session_history table with a composite key includes sessionId as the partition key, and itemInSession as the clustering column.

In [9]:
# Create session_history table that holds data for query 1:
session.execute("""
    CREATE TABLE IF NOT EXISTS session_history (
        sessionId INT, 
        itemInSession INT, 
        artist TEXT, 
        song TEXT, 
        length FLOAT, 
        PRIMARY KEY(sessionId, itemInSession))
""")

In [11]:
# Insert data from df to table session_history:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO session_history (sessionId, itemInSession, artist, song, length) \
                 VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [12]:
# Test and verify the data was entered into the session_history table 
session.row_factory = pandas_dataframe

query = session.execute("""
    SELECT artist, song, length
    FROM session_history 
    WHERE sessionId = 338 AND itemInSession = 4
    """)

df = query._current_rows 
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

For this query, we will create the user_songs table to answer the question using a composite key that made up of userId and sessionId as partition keys, and itemInSession as clustering column in order to sort by itemInSession

In [13]:
# Create user_songs table that holds data for query 2:
session.execute("""
    CREATE TABLE IF NOT EXISTS user_songs (
        userId INT, 
        sessionId INT, 
        itemInSession INT,
        artist TEXT, 
        song TEXT, 
        firstName TEXT, 
        lastName TEXT,
        PRIMARY KEY((userId, sessionId), itemInSession))
""")

In [14]:
# Insert data from df to table user_songs:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO user_songs (userId, sessionId, itemInSession, artist, song, firstName, lastName) \
                 VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [15]:
# Test and verify the data was entered into the user_songs table 
session.row_factory = pandas_dataframe

query = session.execute("""
    SELECT artist, song, firstName, lastName
    FROM user_songs
    WHERE userId=10 AND sessionId=182
    ORDER BY itemInSession;
""")

df = query._current_rows 
df

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

For this query, we will create app_history table that has a composite key made up of song as the partition key, and userId as the clustering colu

In [16]:
# Create app_history table that holds data for query 3:
session.execute("""
    CREATE TABLE IF NOT EXISTS app_history (
        song TEXT,
        userId INT, 
        firstName TEXT, 
        lastName TEXT, 
        PRIMARY KEY(song, userId))
""")

In [17]:
# Insert data into app_history table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO app_history (song, userId, firstName, lastName) \
                 VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [18]:
# Test and verify the data was entered into the user_songs table 
session.row_factory = pandas_dataframe

query = session.execute("""
    SELECT firstName, lastName
    FROM app_history 
    WHERE song = 'All Hands Against His Own'
""")

df = query._current_rows
df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [19]:
# Drop the table before closing out the sessions
session.execute("""DROP TABLE IF EXISTS session_history""")

session.execute("""DROP TABLE IF EXISTS user_songs""")

session.execute("""DROP TABLE IF EXISTS app_history""")

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()